# Surface visualization in 3D
Authors/Copyright: Masa Prodanovic and James McClure, all right reserved.

Update: October 2020

## Motivation

Segmented (also sometimes binary) images are those where each pixel or voxel has been classified as a member of a finite number of groups (phases, physical objects, elements or whatever is of interest to identify).

We will here not spend more on the subject of segmentation (__extremely important one!__), but will use one such segmented image to plot surfaces between different phases(e.g. fluids, mineral phases in subsurface porous media) has already been made. 

In 3D, it is very useful to draw surface that separates two phases or it is placed at an interpolated "level" of data. It is also called isosurface.

## Useful functions for download and statistics

In [5]:
import requests # library for file download

import numpy as np
import matplotlib.pyplot as plt
    
def download_file_url(file_url,filename):

    # download file
    r = requests.get(file_url, stream = True) 

    with open(filename,"wb") as f: 
        for chunk in r.iter_content(chunk_size=1024): 

             # writing one chunk at a time to pdf file 
             if chunk: 
                 f.write(chunk)
    return()



def image_statistics(image,plot_histogram=False):
    print('\nImage (ndarray) shape',image.shape)
    print('\nImage data type',image.dtype)

    # This is a 2D image, and min, max and mean functions are programmed to work by dimension. 
    # So we will reshape the array. 
    # Be mindful that below we are on purpose creating another copy of the array (and doubles the memory use)
    dims = image.shape;
    reshaped_image = np.copy(image) 
    reshaped_image.shape = (np.prod(dims),)

    # min, max, mean are functions that are built into ndarray class in NumPy. If we call them such as in example below,
    # then they will execute on the ndarray reshaped_image
    print('\nImage values min:', reshaped_image.min(), 'max:',reshaped_image.max(), 'mean:',reshaped_image.mean())

    if plot_histogram:
        #Histogram
        # histogram has 256 bins, is normalized (density=True),
        # so that comparison with other images of different size is possible and is shown in green color
        plt.hist(reshaped_image, 256, density=True, facecolor='g', alpha=0.75)

        plt.xlabel('Image value')
        plt.ylabel('Probability')

        #plt.axis([40, 160, 0, 0.03])
        plt.grid(True)
        plt.show()

    return()


# Example from DRP

We will download Digital Rocks Portal Example for the purpose from
https://www.digitalrocksportal.org/projects/125/analysis_data/187/


In [17]:
# File download

file_url = "https://www.digitalrocksportal.org/projects/125/images/101255/download/"
filename = "Ketton_segmented_oil_blob.raw"

download_file_url(file_url,filename)

()

In [34]:
# Read downloaded file.

# the file is binary (.raw) file, 
# and requires information from the webpage on how to read it.
width  = 365
height = 255
slices = 225

# Alternative
#dtype  = 'u1'
#byte = '<'  # little endian byte order
# read date from file
#datatype = byte + dtype    

alldata = np.fromfile(filename, dtype=np.ubyte, sep="")
image = alldata.reshape([slices, height, width])

## Image rescaling
We rescale image to a smaller one in this exercise: this is useful for larger images where surface rendering might take a long time.

In [19]:
from mayavi import mlab
import skimage.transform

print('Original image stats\n')
image_statistics(image, plot_histogram=False)

# need to downsample image since it is too large
# Note - if I provide integer image below, then rescaling 
# shifts the image values even if I preserve_range (i.e. set it to True). 
# Hence I do a type conversion (image.astype(float))
# Likely culprit:
# Some online sites warn that skimage converts all floats to [-1,1]
# and many functions return float even if they get integer array.
image_rescaled  = skimage.transform.rescale(image.astype(float), 0.5, preserve_range=True)

# check stats on rescaled image to make sure the values are preserved
image_statistics(image_rescaled, plot_histogram=False)

# resizing has the same trouble with integer array input, 
# so would have to use image.astype(float) conversion

#factor = 2
#image_resized = skimage.transform.resize(image,[width/factor,height/factor, slices/factor])
#image_statistics(image_resized,plot_histogram=False)

Original image stats


Image (ndarray) shape (225, 255, 365)

Image data type uint8

Image values min: 1 max: 3 mean: 2.7589871967051662

Image (ndarray) shape (112, 128, 182)

Image data type float64

Image values min: 0.9999999999999998 max: 2.999999999999999 mean: 2.758989161169739


()

In [41]:
mlab.volume_slice(image, plane_orientation='x_axes', colormap='gray')
mlab.show()

## Plot just oil blob

In the segmented image, we have 1, 2 and 3 represent oil, brine and rock respectively.

The interpolated value of 1.5 separated oil from everything else.

Thus the surface passing through points where interpolated image values 
are 2.5 is a surface that separates both fluids and the rock.

In [33]:
from mayavi import mlab

mlab.figure(bgcolor=(1,1,1),size=(800,800))
mlab.clf()

iso_oil=1.5
mlab.contour3d(image,contours = [iso_oil],colormap='hot',opacity=1)
mlab.show()

## Plot oil blob and grain surface

In [28]:
from mayavi import mlab

mlab.figure(bgcolor=(1,1,1),size=(800,800))
mlab.clf()

iso_grain = 2.5
iso_oil=1.5
mlab.contour3d(image_rescaled,contours = [iso_oil,iso_grain],opacity=0.25,colormap="hot")
mlab.outline(color=(0,0,0))
mlab.show()